In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as st
from scipy.optimize import minimize
import ipywidgets as widgets
from IPython.display import display,clear_output
from matplotlib.colors import to_hex

In [3]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Identification of overabundant alleles in CARLIN barcoded embryos

In [4]:
def generate_bc_dist():
    num_samples = 1000
    prob_high = 0.02
    randoms = st.expon.rvs(loc=0, scale=10**5, size=int(num_samples*prob_high))
    randoms = np.append(randoms, np.maximum(1,st.norm.rvs(loc=10, scale=10, size=int(num_samples*(1-prob_high)))))
    randoms = -np.sort(-randoms)
    return(randoms)

init_prob_twice_in_mouse = 0.05
init_mouse_threshold = 2
init_num_mice = 5
init_num_cells = 10000

bc_dist = generate_bc_dist()
bc_dist_fig = plt.figure(figsize=(15,5))
bc_dist_ax = bc_dist_fig.add_subplot(121)
bc_pdf_ax = bc_dist_fig.add_subplot(122)
bc_output = widgets.Output()

def calculate_fp_fn():
    def score_poisson_cdf(lam):
        return((st.poisson.sf(1,lam)-prob_twice_in_mouse.value)**2)
    mouse_pois_intensity = minimize(score_poisson_cdf, 1, method='L-BFGS-B', bounds=[(0, 1)]).x[0]
    mouse_threshold = mouse_threshold_input.value
    num_mice = num_mice_input.value
    bc_freqs = bc_dist/np.sum(bc_dist)
    true_neg = bc_freqs[bc_freqs<mouse_pois_intensity/num_cells_input.value]
    true_pos = bc_freqs[bc_freqs>mouse_pois_intensity/num_cells_input.value]
    fn_rate = np.sum([st.binom.cdf(mouse_threshold-1, num_mice, 1-(1-x)**num_cells_input.value) for x in true_pos])/len(true_pos)
    fp_rate = np.sum([st.binom.sf(mouse_threshold-1, num_mice, 1-(1-x)**num_cells_input.value) for x in true_neg])/len(true_neg)
    return((fn_rate, fp_rate))

def generate_text_fp_fn():
    fn_rate, fp_rate = calculate_fp_fn()
    if fn_rate < 0.01:
        fn_to_print = '{:0.3e}'.format(fn_rate)
    else:
        fn_to_print = str(round(fn_rate,2))
    if fp_rate < 0.01:
        fp_to_print = '{:0.3e}'.format(fp_rate)
    else:
        fp_to_print = str(round(fp_rate,2))
    
    hex_color = str(to_hex("xkcd:bright purple"))
    fp_to_print = '<b><font color="'+hex_color+'">' + fp_to_print + "</font></b>"
    fn_to_print = '<b><font color="'+hex_color+'">' + fn_to_print + "</font></b>"
    return("False negative rate: "+fn_to_print+"; false positive rate: "+fp_to_print)

def plot_bc_size(threshold):
    bc_dist_ax.clear()
    cmap = {True: 'blue', False:"red"}
    bc_dist_ax.bar(np.arange(len(bc_dist)), np.log10(bc_dist), color=[cmap[x] for x in bc_dist < (np.sum(bc_dist)*threshold)])
    bc_dist_ax.set_xlabel('sorted barcode ID')
    bc_dist_ax.set_ylabel('log10 number of transcripts for each barcode')
    
def plot_bc_pdf(threshold):
    bc_pdf_ax.clear()
    
    lower = bc_dist[bc_dist < (np.sum(bc_dist)*threshold)]
    upper = bc_dist[bc_dist > (np.sum(bc_dist)*threshold)]
    bc_pdf_ax.hist(np.log10(upper), color="red")
    bc_pdf_ax.hist(np.log10(lower), color="blue")
    bc_pdf_ax.set_xlabel('log10 number of transcripts for each barcode')
    bc_pdf_ax.set_ylabel('frequency')
    
def mouse_listener(change):
    fp_fn_text.value = generate_text_fp_fn()

def threshold_plot_listener(change):
    def score_poisson_cdf(lam):
        return((st.poisson.sf(1,lam)-change['new'])**2)
    mouse_pois_intensity = minimize(score_poisson_cdf, 1, method='L-BFGS-B', bounds=[(0, 1)]).x[0]
    threshold = mouse_pois_intensity/num_cells_input.value
    plot_bc_size(threshold)
    plot_bc_pdf(threshold)
    fp_fn_text.value = generate_text_fp_fn()
    with bc_output:
        clear_output(wait=True)
        display(bc_dist_fig)

def score_poisson_cdf(lam):
    return((st.poisson.sf(1,lam)-init_prob_twice_in_mouse)**2)
mouse_pois_intensity = minimize(score_poisson_cdf, 1, method='L-BFGS-B', bounds=[(0, 1)]).x[0]
init_threshold = mouse_pois_intensity/init_num_cells
plot_bc_size(init_threshold)
plot_bc_pdf(init_threshold)
with bc_output:
    clear_output(wait=True)
    display(bc_dist_fig)

prob_twice_in_mouse = widgets.BoundedFloatText(
    value=init_prob_twice_in_mouse,
    min=0,
    max=1.0,
    step=0.01,
    description='',
    disabled=False,
    continuous_update=False
)

num_cells_input = widgets.BoundedIntText(
    value=init_num_cells,
    min=0,
    max=1e40,
    description='',
    disabled=False,
    continuous_update=False
)

mouse_threshold_input = widgets.BoundedIntText(
    value=init_mouse_threshold,
    min=0,
    max=1e40,
    description='',
    disabled=False,
    continuous_update=False
)

num_mice_input = widgets.BoundedIntText(
    value=init_num_mice,
    min=0,
    max=10000,
    description='',
    disabled=False,
    continuous_update=False
)

fp_fn_text = widgets.HTML(
    value=generate_text_fp_fn(),
    description='',
    layout=widgets.Layout(width='100%', height="100px")
)
        
prob_twice_in_mouse.observe(threshold_plot_listener, names='value')
num_mice_input.observe(mouse_listener, names='value')
mouse_threshold_input.observe(mouse_listener, names='value')
num_cells_input.observe(threshold_plot_listener, names='value')
plt.close()

In [5]:
prob_twice_text = widgets.HTML(
    value='Threshold probability of generating two or more of a particular barcode independently in a mouse:',
    disabled=False
)
prob_twice_box = widgets.HBox([prob_twice_text, prob_twice_in_mouse])

num_cells_text = widgets.HTML(
    value='Number of cells per mouse in which a barcode is induced:',
    disabled=False
)
num_cells_box = widgets.HBox([num_cells_text, num_cells_input])

mouse_threshold_text = widgets.HTML(
    value='Mouse number threshold for calling overabundant barcodes:',
    disabled=False
)
mouse_threshold_box = widgets.HBox([mouse_threshold_text, mouse_threshold_input])

num_mice_text = widgets.HTML(
    value='Total number of mice used:',
    disabled=False
)
num_mice_box = widgets.HBox([num_mice_text, num_mice_input])

display(prob_twice_box, num_cells_box, mouse_threshold_box, num_mice_box, bc_output, fp_fn_text)

Output()

HTML(value='False negative rate: <b><font color="#be03fd">1.472e-12</font></b>; false positive rate: <b><font …

## A bit of explanation...

Our goal is to create a procedure than can accurately identify overabundant barcodes that should be removed from downstream analysis because they are too likely to be generated independently multiple times in the same mouse. We can control how stringently we define "overabundance" of a barcode by specifying a cutoff value for the probability that a particular barcode will be generated multiple times in a mouse (first input box above). If this value is 0.05, for example, barcodes that are expected to be independently induced multiple times in more than 5% of mice will be excluded from further analysis.

In the plots above, red areas represent barcodes that *should* be marked as overabundant, because their probability of being generated more than once in the same mouse is higher than the given threshold probability. Similarly, blue areas indicate barcodes that are unlikely to be generated more than once. The barcode frequency plots here are artificially generated to appear similar to the CARLIN granulocyte allele bank; in particular, the left plot should look similar to the gray area in Supplementary Figure 3C of that paper.

To identify these overabundant barcodes, our general strategy is to sequence barcodes from multiple mice ("total number of mice used" above)- barcodes that appear in several mice (more mice than the given threshold value above) are classifed as overabundant. Unfortunately, due to randomness in barcode appearance, we will misclassify some of the barcodes. This error can be quantified using the false negative rate (probability that an overabundant barcode will be misclassified as ok to use in downstream analysis) and the false positive rate (probability that a barcode that is unlikely to come up more than once in a mouse is misclassified as overabundant). The false negative and positive rates are shown above in purple.

### Assumptions
+ The allele frequency distribution immediately after induction is known.
+ No alleles are lost between induction and sequencing (through cell death and/or sampling loss).